In [1]:
import random
import math
import numpy as np
import copy
import getopt
import psycopg2
import sys
import time
import matplotlib.pyplot as plt

In [2]:
def LapNoise():
    a = random.uniform(0,1)
    b = math.log(1/(1-a))
    c = random.uniform(0,1)
    if c>0.5:
        return b
    else:
        return -b
def count(D, a, b):
    counter = 0
    n = len(D)
    for i in range(n):
        if D[i] >=a and D[i]<=b:
            counter+=1
    return counter
def unboundedDomainRadius(eps, beta, D, B):
    TNoise = 2/eps*LapNoise()
    T = len(D)- 6/eps*np.log(2/beta)+TNoise
#     print(T)
    for i in range(int(math.log2(B))):
        Qi = count(D,0, math.pow(2 ,i))+4/eps*LapNoise()
#         print(Qi)
        if Qi>T:
            break
    return  math.pow(2,i-1)   

    
def clip(D, a, b):
    clipped = copy.deepcopy(D)
    clipped[clipped<a] = a
    clipped[clipped>b] = b
    return clipped

def universal(I, eps, B, beta):
    rad = unboundedDomainRadius(eps/2, beta/2, I, B)
#     print(rad)
    clipped = clip(I,0,rad)
    
    return sum(clipped)+ 2*rad/(eps)*LapNoise()

In [20]:
num_user=[1000000, 1000000, 1000000, 1000000, 1000000]
ratio = 0.1
num_repeats = 1
B = [100000, 1000000, 10000000, 100000000, 1000000000]
ratio = 0.1
fc = 0.54
fm = 0.37
fl = 0.09


In [13]:
def sample(eps, I, n, t, B, beta=0.1):
    temp = copy.deepcopy(I)
    for i in range(n):
        if eps[i]<t:
            temp[i] = np.random.binomial(1, (math.exp(eps[i])-1)/(math.exp(t)-1))*I[i]
            
    return universal(temp, t, B, beta)

In [22]:
times = []
errors = []
relative_error = []
real_results = []
#should be changed to range(5) in simluated datasets
for i in range(5):
    #uncomment this line in simluated datasets
    n = int(num_user[i]*ratio)
    
    #Gaussian eps
#     #eps of non-zero users
#     eps = np.random.normal(1,0.3,int(n))
#     eps[eps<1/(10*math.sqrt(num_user[i]))]=1/(math.sqrt(10*num_user[i]))

#     # t = sum(eps)/n
#     eps_sort = np.sort(eps)
#     #to enable EM, we also need eps for zero users
#     eps_zero = np.random.normal(1,0.3,num_user[i]-int(n))
#     eps_zero[eps_zero<1/(10*math.sqrt(num_user[i]))]=1/(10*math.sqrt(num_user[i]))
#     eps_zero_sort = np.sort(eps_zero)

    
#     #Uniform eps
    nc = int(n*fc)
    nm = int(n*fm)
    nl = int(n*fl)
    epsc = np.random.uniform(1/(10*math.sqrt(num_user[i])), 0.5, nc)
    epsm = np.random.uniform(0.5, 2, nm)
    epsl = np.ones(nl)*2
    eps = np.append(np.append(epsc, epsm), epsl)
    eps_sort = np.sort(eps)
    #to enable EM, we also need eps for zero users
    nc_z = int((num_user[i]-n)*fc)
    nm_z = int((num_user[i]-n)*fm)
    nl_z = int((num_user[i]-n)*fl)
    epsc_z = np.random.uniform(1/(10*math.sqrt(num_user[i])), 0.5, nc_z)
    epsm_z = np.random.uniform(0.5, 2, nm_z)
    epsl_z = np.ones(nl_z)*2
    eps_zero = np.append(np.append(epsc_z, epsm_z), epsl_z)
    eps_zero_sort = np.sort(eps_zero)
    
    t = (sum(eps)+sum(eps_zero))/num_user[i]
    eps_min = min(eps_sort[0], eps_zero_sort[0])
    eps_max = max(eps_sort[n-1], eps_zero_sort[9*n-1])
    
    I = np.random.normal(1000, 200, n)
    I = np.int_(I)
    I[I<=0]=1
    real  = sum(I)
#     print(real)
    real_results.append(real)
    
    start = time.time()
    error = 0
    for j in range(num_repeats):
        result = sample(eps, I, n, t, B[i])
        error += abs(result-real)
    
    
    end= time.time()
    times.append((end-start)/num_repeats)
    errors.append(error/num_repeats)
    relative_error.append((error/num_repeats)/real)
#     print("Average Time for i = "+str(i))
#     print((end-start)/num_repeats)
#     print("Average Error for i = "+str(i))
#     print(error/num_repeats)

    
print("times")
print(times)
print("errors")
print(errors)
print("relative_error")
print(relative_error)
    


times
[0.8452510833740234, 0.8523690700531006, 0.8325042724609375, 0.8498959541320801, 0.8380510807037354]
errors
[46004462.46152174, 46056768.31150468, 45846166.97158251, 45792172.425570056, 45963815.699798994]
relative_error
[0.4599156136864222, 0.46054576113681667, 0.458562727769771, 0.45814733432901755, 0.45946136086094425]


In [23]:
def pdp_sum(eps, I, n, eps_min, eps_max, beta, B):
    t = math.ceil(math.log(eps_max*B/eps_min, 2))
    result=0
    for i in range(t+1):
        tau = np.zeros(n)
        temp = copy.deepcopy(I)
        for j in range(n):
            tau[j] = math.floor(math.pow(2, i)*eps[j]/eps_max)
            temp[j] = min(tau[j], temp[j])
#         print("i = "+str(i))
        sum_tilde = sum(temp)+ t*math.pow(2, i)/eps_max*LapNoise()-t*math.pow(2, i)/eps_max*np.log(t/beta)
#         print(sum_tilde)
        result = max(result,sum_tilde)
    return result

In [24]:
times = []
errors = []
relative_error = []
real_results = []
for i in range(5):
    n = int(num_user[i]*ratio)

    #Gaussian eps
#     #eps of non-zero users
#     eps = np.random.normal(1,0.3,int(n))
#     eps[eps<1/(10*math.sqrt(num_user[i]))]=1/(math.sqrt(10*num_user[i]))

#     # t = sum(eps)/n
#     eps_sort = np.sort(eps)
#     #to enable EM, we also need eps for zero users
#     eps_zero = np.random.normal(1,0.3,num_user[i]-int(n))
#     eps_zero[eps_zero<1/(10*math.sqrt(num_user[i]))]=1/(10*math.sqrt(num_user[i]))
#     eps_zero_sort = np.sort(eps_zero)
    
#     #Uniform eps
    nc = int(n*fc)
    nm = int(n*fm)
    nl = int(n*fl)
    epsc = np.random.uniform(1/(10*math.sqrt(num_user[i])), 0.5, nc)
    epsm = np.random.uniform(0.5, 2, nm)
    epsl = np.ones(nl)*2
    eps = np.append(np.append(epsc, epsm), epsl)
    eps_sort = np.sort(eps)
    #to enable EM, we also need eps for zero users
    nc_z = int((num_user[i]-n)*fc)
    nm_z = int((num_user[i]-n)*fm)
    nl_z = int((num_user[i]-n)*fl)
    epsc_z = np.random.uniform(1/(10*math.sqrt(num_user[i])), 0.5, nc_z)
    epsm_z = np.random.uniform(0.5, 2, nm_z)
    epsl_z = np.ones(nl_z)*2
    eps_zero = np.append(np.append(epsc_z, epsm_z), epsl_z)
    eps_zero_sort = np.sort(eps_zero)
    
    t = (sum(eps)+sum(eps_zero))/num_user[i]
    eps_min = min(eps_sort[0], eps_zero_sort[0])
    eps_max = max(eps_sort[n-1], eps_zero_sort[9*n-1])
    
    I = np.random.normal(1000, 200, n)
    I = np.int_(I)
    I[I<=0]=1
#     real  = sum(I)
    real_results.append(real)
    
    start = time.time()
    error = 0
#     print(real)
    for j in range(num_repeats):
        result = pdp_sum(eps, I, n, eps_min, eps_max, 0.1, B[i])

        error += abs(result-real)
#         print(abs(result-real))
    
    
    end= time.time()
    times.append((end-start)/num_repeats)
    errors.append(error/num_repeats)
    relative_error.append((error/num_repeats)/real)
#     print("Average Time for i = "+str(i))
#     print((end-start)/num_repeats)
#     print("Average Error for i = "+str(i))
#     print(error/num_repeats)

    
print("times")
print(times)
print("errors")
print(errors)
print("relative_error")
print(relative_error)
    


times
[1.581805944442749, 1.7114348411560059, 1.8598589897155762, 2.0289878845214844, 2.2211151123046875]
errors
[6876551.3463386595, 6752274.21950528, 7860480.465166181, 8086069.287915796, 7342491.707652926]
relative_error
[0.0687390633591966, 0.06749677011288106, 0.07857456994289348, 0.08082959046104446, 0.0733966747700445]


In [25]:
times = []
errors = []
relative_error = []
real_results = []
for i in range(5):
    n = int(num_user[i]*ratio)

    #Gaussian eps
#     #eps of non-zero users
#     eps = np.random.normal(1,0.3,int(n))
#     eps[eps<1/(10*math.sqrt(num_user[i]))]=1/(math.sqrt(10*num_user[i]))

#     # t = sum(eps)/n
#     eps_sort = np.sort(eps)
#     #to enable EM, we also need eps for zero users
#     eps_zero = np.random.normal(1,0.3,num_user[i]-int(n))
#     eps_zero[eps_zero<1/(10*math.sqrt(num_user[i]))]=1/(10*math.sqrt(num_user[i]))
#     eps_zero_sort = np.sort(eps_zero)
    
#     #Uniform eps
    nc = int(n*fc)
    nm = int(n*fm)
    nl = int(n*fl)
    epsc = np.random.uniform(1/(10*math.sqrt(num_user[i])), 0.5, nc)
    epsm = np.random.uniform(0.5, 2, nm)
    epsl = np.ones(nl)*2
    eps = np.append(np.append(epsc, epsm), epsl)
    eps_sort = np.sort(eps)
    #to enable EM, we also need eps for zero users
    nc_z = int((num_user[i]-n)*fc)
    nm_z = int((num_user[i]-n)*fm)
    nl_z = int((num_user[i]-n)*fl)
    epsc_z = np.random.uniform(1/(10*math.sqrt(num_user[i])), 0.5, nc_z)
    epsm_z = np.random.uniform(0.5, 2, nm_z)
    epsl_z = np.ones(nl_z)*2
    eps_zero = np.append(np.append(epsc_z, epsm_z), epsl_z)
    eps_zero_sort = np.sort(eps_zero)
    
    t = (sum(eps)+sum(eps_zero))/num_user[i]
    eps_min = min(eps_sort[0], eps_zero_sort[0])
    eps_max = max(eps_sort[n-1], eps_zero_sort[9*n-1])
    I = np.random.normal(1000, 200, n)
    I = np.int_(I)
    I[I<=0]=1
#     real  = sum(I)
    real_results.append(real)
    
    start = time.time()
    error = 0
    for j in range(num_repeats):
        result = universal(I, eps_min, B[i], 0.1)

        error += abs(result-real)
    
    
    end= time.time()
    times.append((end-start)/num_repeats)
    errors.append(error/num_repeats)
    relative_error.append((error/num_repeats)/real)
#     print("Average Time for i = "+str(i))
#     print((end-start)/num_repeats)
#     print("Average Error for i = "+str(i))
#     print(error/num_repeats)

    
print("times")
print(times)
print("errors")
print(errors)
print("relative_error")
print(relative_error)

times
[0.07072806358337402, 0.073944091796875, 0.06925511360168457, 0.06987714767456055, 0.06833600997924805]
errors
[100028263.40359133, 100025986.81194861, 100038090.35925737, 100040059.52239352, 100046452.77135606]
relative_error
[0.999897883329387, 0.9998751261696873, 0.9999961150874492, 1.0000157991445824, 1.0000797070430876]
